## Import

In [1]:
%load_ext autoreload
%autoreload 2
import logging
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
from pylab import rcParams

import core.config as cfg
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as pri
import vendors.kibot.utils as kut
import vendors.particle_one.PartTask269_liquidity_analysis_utils as lau

sns.set()


rcParams["figure.figsize"] = (20, 5)

In [3]:
print(env.get_system_signature())

pri.config_notebook()

# TODO(gp): Changing level during the notebook execution doesn't work. Fix it.
# dbg.init_logger(verb=logging.DEBUG)
dbg.init_logger(verb=logging.INFO)
# dbg.test_logger()

_LOG = logging.getLogger(__name__)

# Packages
         python: 3.6.7
         joblib: 0.13.2
          numpy: 1.17.2
         pandas: 0.25.1
        pyarrow: 0.14.1
          scipy: 1.3.1
        seaborn: 0.9.0
        sklearn: 0.21.3
    statsmodels: 0.10.1
# Last commits:
  *   259b518 Julia    Merge branch 'master' into PartTask269_PRICE_Liquidity_analysis_as_a_proxy_for_trader_interest (    6 days ago) Fri Oct 4 10:47:46 2019  (HEAD -> PartTask269_PRICE_Liquidity_analysis_as_a_proxy_for_trader_interest, origin/PartTask269_PRICE_Liquidity_analysis_as_a_proxy_for_trader_interest)
  |\  
  | * fc9b2ee paul     Only expose a reader for the sample data. Factor out some common code. (    7 days ago) Thu Oct 3 19:39:26 2019           
  | * 56cd554 paul     Add 324K of SPY sample SPY data.                                  (    7 days ago) Thu Oct 3 19:27:22 2019           


# Metadata

In [6]:
df1 = kut.read_metadata1()
df1.head(3)

s3://alphamatic/kibot/metadata/All_Futures_Contracts_1min.csv.gz


AssertionError: 

In [ ]:
df2 = kut.read_metadata2()
df2.head(3)

In [ ]:
df3 = kut.read_metadata3()
df3.head(3)

In [ ]:
df4 = kut.read_metadata4()
print(df4.head(3))

print(df4["Exchange"].unique())

# Load product specs

In [ ]:
# TODO (Julia): After PartTask268_PRICE_Download_metadata_from_CME
# is merged into master, replace this with a reader
_PRODUCT_SPECS_PATH = (
    "/data/prices/product_slate_export_with_contract_specs_20190905.csv"
)
product_specs = pd.read_csv(_PRODUCT_SPECS_PATH)

In [ ]:
product_specs.head()

In [ ]:
product_specs.info()

# Explore metadata

In [ ]:
df4["Exchange"].value_counts()

In [ ]:
df3["Exchange"].value_counts()

Kibot only has the CME group futures.

In [ ]:
product_specs["Globex"].head()

In [ ]:
daily_futures_w_ext = os.listdir(
    "/data/kibot/All_Futures_Continuous_Contracts_daily/"
)

In [ ]:
daily_futures_w_ext[:5]

In [ ]:
daily_futures = list(map(lambda x: x[:-7], daily_futures_w_ext))
daily_futures[:5]

In [ ]:
len(set(daily_futures)), df3["SymbolBase"].nunique()

In [ ]:
np.setdiff1d(df3["SymbolBase"].dropna().values, daily_futures)

In [ ]:
product_specs["Globex"].nunique()

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df3["SymbolBase"].dropna().values
)

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df3["SymbolBase"].dropna().values
).shape

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df2["Symbol"].dropna().values
).shape

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df1["Symbol"].dropna().values
).shape

In [ ]:
product_specs[product_specs["Globex"].isna()]

In [ ]:
mask = ["GAS" in d or "OIL" in d for d in df4["Description"].astype(str)]
print(sum(mask))
print(df4[mask].drop(["SymbolBase", "Size(MB)"], axis=1))

In [ ]:
df4[mask]["Symbol"].values

# Read config

In [ ]:
config = cfg.Config.from_env()

if config is None:
    config = cfg.Config()
    config_tmp = config.add_subconfig("read_data")
    # config_tmp["nrows"] = 100000
    config_tmp["nrows"] = None
    #
    config["zscore_com"] = 28

print(config)

# Prices

## Read daily prices

In [ ]:
all_symbols = [
    futures.replace(".csv.gz", "")
    for futures in os.listdir(
        "/data/kibot/All_Futures_Continuous_Contracts_daily"
    )
]

In [ ]:
symbols = df4[mask]["Symbol"].values
symbols

In [ ]:
file_name = "/data/kibot/All_Futures_Continuous_Contracts_daily/%s.csv.gz"

daily_price_dict_df = kut.read_multiple_symbol_data(
    symbols, file_name, nrows=config["read_data"]["nrows"]
)

daily_price_dict_df["CL"].tail(2)

# Top futures by volume

## Sum volume

In [ ]:
daily_vol = lau.get_sum_daily_volume(daily_price_dict_df)
daily_vol.sort_values("sum_vol", ascending=False)

## Mean volume

In [ ]:
mean_vol = lau.get_mean_daily_volume(daily_price_dict_df)
mean_vol.sort_values("mean_vol", ascending=False)

# Study volume

In [ ]:
symbol = "CL"

In [ ]:
vs = lau.VolumeStudy(symbol, n_rows=None)

In [ ]:
vs.execute()

## How is the volume related to the open interest from the metadata?

In [ ]:
product_specs.head()

In [ ]:
product_specs[product_specs["Globex"] == symbol]["Open Interest"].values

In [ ]:
product_specs[product_specs["Globex"] == symbol]["Volume"].values

In [ ]:
vs.daily_prices["vol"].max()

In [ ]:
vs.minutely_prices["vol"].max()

# CME mapping

## Groups overview

In [ ]:
pc = lau.ProductSpecs()

In [ ]:
pc.product_specs.info()

In [ ]:
pc.product_specs["Product Group"].value_counts().plot(kind="bar", rot=0)
plt.title("Number of futures for each product group in CME")
plt.show()

In [ ]:
pc.product_specs["Sub Group"].value_counts().plot(kind="bar")
plt.xticks(ha="right", rotation=30, rotation_mode="anchor")
plt.title("Number of futures for each sub group in CME")
plt.show()

In [ ]:
pc.product_specs["Category"].astype(str).value_counts()

In [ ]:
pc.product_specs["Sub Category"].astype(str).value_counts()

## By symbol

In [ ]:
pc.get_metadata_symbol(symbol)

In [ ]:
pc.get_product_group(symbol)

In [ ]:
pc.get_trading_hours(symbol)

## For product group

In [ ]:
energy_symbols = pc.get_symbols_product_group("Energy")
energy_symbols[:4]

In [ ]:
np.intersect1d(energy_symbols, daily_futures)

In [ ]:
np.intersect1d(energy_symbols, daily_futures).shape